# List of users report

### Imports

In [1]:
import sys
import psycopg2
import pandas as pd
from datetime import datetime

### Initial setup

In [2]:
conn = psycopg2.connect(host='localhost',
                        port='5432',
                        user='mffais',
                        password='pass',
                        database='bigquery')

### Fetch users

In [3]:
cursor = conn.cursor()
query = '''
    SELECT user_pseudo_id AS user_id,
           geo ->> 'country' AS country,
           traffic_source ->> 'source' AS source,
           traffic_source ->> 'name' AS campaign,
           DATE(user_first_touch_timestamp) AS install_date
    FROM events
    WHERE geo ->> 'country' <> ''
      AND user_first_touch_timestamp IS NOT NULL
    GROUP BY user_pseudo_id, country, source, campaign, user_first_touch_timestamp
    ORDER BY country
'''
cursor.execute(query)
list_of_users = pd.read_sql(query, con=conn)
cursor.close()
list_of_users.rename(columns={ 'user_id':'User ID',
                               'country':'Country',
                                'source':'Source',
                              'campaign':'Campaign',
                          'install_date':'Install date'}, inplace=True)
list_of_users.head(20)

,User ID,Country,Source,Campaign,Install date
0,0105b05330d31f59ca038f0d1d8da7bd,Afghanistan,google,New App Promotion,2019-10-19
1,0a85b5a7c6dba28c3ecc67838e445739,Afghanistan,google,New App Promotion,2019-10-17
2,12279f01f524bc563bfce313b3eac7aa,Afghanistan,google,New App Promotion,2019-10-19
3,15e6cad83ec6c0832abb0c966c9e2af2,Afghanistan,google,New App Promotion,2019-10-15
4,2047558519cf8e31aeaed984d7a8f2ff,Afghanistan,google,New App Promotion,2019-10-21
5,2859707815c480426d86e8da9ab9e79d,Afghanistan,google,kevin-auto-savings 50c,2019-10-26
6,2fa7d86eea6c6018983a7d9475f1bf08,Afghanistan,google,New App Promotion,2019-10-20
7,35f4061076083c79bb40e25e9753d6a9,Afghanistan,google,kevin-minimalist 50c,2019-10-26
8,3cb92ed8b096137315da3fab0d12f5fe,Afghanistan,google,kevin-privacy-50c,2019-10-28
9,41432b003c68b75fd946be5f7296fbfc,Afghanistan,google,New App Promotion,2019-10-19


### Calculate uninstalls

In [4]:
cursor = conn.cursor()
query = '''
    SELECT user_pseudo_id AS user_id,
           DATE(event_timestamp) AS uninstall_date
    FROM events
    WHERE event_name='app_remove'
'''
cursor.execute(query)
uninstall = pd.read_sql(query, con=conn)
cursor.close()
uninstall.rename(columns={ 'user_id':'User ID', 'uninstall_date':'Uninstall date' }, inplace=True)
uninstall.head(20)

,User ID,Uninstall date
0,36436e319b5fbfae1d2a34f69211dbee,2019-10-12
1,c69438230d7b866dd7b11c6207d79312,2019-10-13
2,6bc7e5be278fbbfad0eaf996b56f43f8,2019-10-14
3,47289e491c9ee2370e687fbeb30e9cb9,2019-10-14
4,ca203c6a1407e748fd721947718828d2,2019-10-14
5,cef764f05e74f4d6354792593ba8f520,2019-10-14
6,1ec5bfa5e9dac4b451ae5cfbaa1ace26,2019-10-13
7,2bc9686109b8ae3427807045802ca087,2019-10-13
8,d396e0e8b10734dfcd3bb62f95a16035,2019-10-14
9,1ae3913d7a79fb9e71268c67a325b0b3,2019-10-13


### Calculate days installed

In [5]:
list_of_users = pd.merge(list_of_users, uninstall, on='User ID', how='left')
list_of_users.head(20)

,User ID,Country,Source,Campaign,Install date,Uninstall date
0,0105b05330d31f59ca038f0d1d8da7bd,Afghanistan,google,New App Promotion,2019-10-19,NaN
1,0a85b5a7c6dba28c3ecc67838e445739,Afghanistan,google,New App Promotion,2019-10-17,NaN
2,12279f01f524bc563bfce313b3eac7aa,Afghanistan,google,New App Promotion,2019-10-19,2019-10-19
3,15e6cad83ec6c0832abb0c966c9e2af2,Afghanistan,google,New App Promotion,2019-10-15,NaN
4,2047558519cf8e31aeaed984d7a8f2ff,Afghanistan,google,New App Promotion,2019-10-21,NaN
5,2859707815c480426d86e8da9ab9e79d,Afghanistan,google,kevin-auto-savings 50c,2019-10-26,2019-10-26
6,2fa7d86eea6c6018983a7d9475f1bf08,Afghanistan,google,New App Promotion,2019-10-20,2019-10-20
7,35f4061076083c79bb40e25e9753d6a9,Afghanistan,google,kevin-minimalist 50c,2019-10-26,NaN
8,3cb92ed8b096137315da3fab0d12f5fe,Afghanistan,google,kevin-privacy-50c,2019-10-28,NaN
9,41432b003c68b75fd946be5f7296fbfc,Afghanistan,google,New App Promotion,2019-10-19,2019-10-26


In [6]:
today = datetime.date( datetime.now() )

def cleanNaN(row):
    uninstall_date = row['Uninstall date']
    if str( uninstall_date ) == 'nan':
        return ''
    return uninstall_date

def date_diff(row):
    install_date = row['Install date']
    if str( row['Uninstall date'] ) == '':
        used_date = today
    else:
        used_date = row['Uninstall date']
    days_installed = int( int( ( used_date - install_date ).total_seconds() ) / 24 / 60 / 60 + 1 )
    return days_installed

list_of_users['Uninstall date'] = list_of_users.apply(cleanNaN, axis=1)
list_of_users['Days installed'] = list_of_users.apply(date_diff, axis=1)

list_of_users.head(20)

,User ID,Country,Source,Campaign,Install date,Uninstall date,Days installed
0,0105b05330d31f59ca038f0d1d8da7bd,Afghanistan,google,New App Promotion,2019-10-19,,14
1,0a85b5a7c6dba28c3ecc67838e445739,Afghanistan,google,New App Promotion,2019-10-17,,16
2,12279f01f524bc563bfce313b3eac7aa,Afghanistan,google,New App Promotion,2019-10-19,2019-10-19,1
3,15e6cad83ec6c0832abb0c966c9e2af2,Afghanistan,google,New App Promotion,2019-10-15,,18
4,2047558519cf8e31aeaed984d7a8f2ff,Afghanistan,google,New App Promotion,2019-10-21,,12
5,2859707815c480426d86e8da9ab9e79d,Afghanistan,google,kevin-auto-savings 50c,2019-10-26,2019-10-26,1
6,2fa7d86eea6c6018983a7d9475f1bf08,Afghanistan,google,New App Promotion,2019-10-20,2019-10-20,1
7,35f4061076083c79bb40e25e9753d6a9,Afghanistan,google,kevin-minimalist 50c,2019-10-26,,7
8,3cb92ed8b096137315da3fab0d12f5fe,Afghanistan,google,kevin-privacy-50c,2019-10-28,,5
9,41432b003c68b75fd946be5f7296fbfc,Afghanistan,google,New App Promotion,2019-10-19,2019-10-26,8


### Calculate sessions since installed

In [7]:
cursor = conn.cursor()
query = '''
    SELECT user_pseudo_id AS user_id,
           COUNT(*) AS sessions
    FROM (
      SELECT event_date,
             user_pseudo_id
      FROM events
      GROUP BY event_date, user_pseudo_id
    ) AS users_by_day
    GROUP BY user_id
'''
cursor.execute(query)
sessions = pd.read_sql(query, con=conn)
sessions.rename(columns={ 'user_id':'User ID', 'sessions':'Sessions' }, inplace=True)
sessions.head(20)

,User ID,Sessions
0,82416a67045a540d8a6d634d87705af4,1
1,30d6318149646f5dd80c598ae26a063f,1
2,c781b9a172d21f20aa113630b51c31e1,1
3,d0ca9c71e106d578db16464e5a79ba5d,1
4,ae57eccdb3c0e96407167f412a63a3ae,1
5,a306edea3da62424693cab2edfce8b29,2
6,73fa067deb27db49bb901a0038e17f11,1
7,a044570ced3441416cfbf1f617b21b0f,2
8,61c107f548ae190979ff0b3fdd718500,1
9,9af305db3b59b2a02063009d76761792,1


In [8]:
list_of_users = pd.merge(list_of_users, sessions, on='User ID', how='left')
list_of_users.head(20)

,User ID,Country,Source,Campaign,Install date,Uninstall date,Days installed,Sessions
0,0105b05330d31f59ca038f0d1d8da7bd,Afghanistan,google,New App Promotion,2019-10-19,,14,1
1,0a85b5a7c6dba28c3ecc67838e445739,Afghanistan,google,New App Promotion,2019-10-17,,16,3
2,12279f01f524bc563bfce313b3eac7aa,Afghanistan,google,New App Promotion,2019-10-19,2019-10-19,1,1
3,15e6cad83ec6c0832abb0c966c9e2af2,Afghanistan,google,New App Promotion,2019-10-15,,18,1
4,2047558519cf8e31aeaed984d7a8f2ff,Afghanistan,google,New App Promotion,2019-10-21,,12,3
5,2859707815c480426d86e8da9ab9e79d,Afghanistan,google,kevin-auto-savings 50c,2019-10-26,2019-10-26,1,1
6,2fa7d86eea6c6018983a7d9475f1bf08,Afghanistan,google,New App Promotion,2019-10-20,2019-10-20,1,1
7,35f4061076083c79bb40e25e9753d6a9,Afghanistan,google,kevin-minimalist 50c,2019-10-26,,7,1
8,3cb92ed8b096137315da3fab0d12f5fe,Afghanistan,google,kevin-privacy-50c,2019-10-28,,5,1
9,41432b003c68b75fd946be5f7296fbfc,Afghanistan,google,New App Promotion,2019-10-19,2019-10-26,8,3


### Output HTTP Header

In [9]:
print('Content-type: text/csv')
print('Content-Disposition: attachment; filename="list_of_users.csv"')
print()

Content-type: text/csv
Content-Disposition: attachment; filename="list_of_users.csv"



### Output variables

In [10]:
print('# Title: List of users report')

# Title: List of users report


### Ouput result

In [11]:
str = list_of_users.to_csv(index=False)
print(str.encode('ascii','xmlcharrefreplace').decode('utf-8'))

User ID,Country,Source,Campaign,Install date,Uninstall date,Days installed,Sessions
0105b05330d31f59ca038f0d1d8da7bd,Afghanistan,google,New App Promotion,2019-10-19,,14,1
0a85b5a7c6dba28c3ecc67838e445739,Afghanistan,google,New App Promotion,2019-10-17,,16,3
12279f01f524bc563bfce313b3eac7aa,Afghanistan,google,New App Promotion,2019-10-19,2019-10-19,1,1
15e6cad83ec6c0832abb0c966c9e2af2,Afghanistan,google,New App Promotion,2019-10-15,,18,1
2047558519cf8e31aeaed984d7a8f2ff,Afghanistan,google,New App Promotion,2019-10-21,,12,3
2859707815c480426d86e8da9ab9e79d,Afghanistan,google,kevin-auto-savings 50c,2019-10-26,2019-10-26,1,1
2fa7d86eea6c6018983a7d9475f1bf08,Afghanistan,google,New App Promotion,2019-10-20,2019-10-20,1,1
35f4061076083c79bb40e25e9753d6a9,Afghanistan,google,kevin-minimalist 50c,2019-10-26,,7,1
3cb92ed8b096137315da3fab0d12f5fe,Afghanistan,google,kevin-privacy-50c,2019-10-28,,5,1
41432b003c68b75fd946be5f7296fbfc,Afghanistan,google,New App Promotion,2019-10-19,2019-10-26,8,3
4525

### Release resources

In [12]:
conn.close()